In [15]:
import os
import cv2
from PIL import Image
import easyocr
import matplotlib.pyplot as plt

In [1]:

# Función para mostrar la imagen
def mostrar_imagen(imagen):
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Inicializar el lector OCR de EasyOCR

def procesar_imagen(imagen):
    imagen_blur = cv2.GaussianBlur( imagen.copy(), (5, 5), 0)
    try: 
        imagen_gray = cv2.cvtColor(imagen_blur, cv2.COLOR_BGR2GRAY)
    except:
        imagen_gray =imagen_blur
    imagen_contrast = cv2.convertScaleAbs(imagen_gray, alpha=1.2, beta=10)
    _, imagen_binaria = cv2.threshold(imagen_contrast, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return imagen_binaria


def detectar_texto(imagen_procesada):
    resultado = lector.readtext(imagen_procesada, detail=1)
    for (bbox, texto, probabilidad) in resultado:
        # Extraer las coordenadas del bounding box
        (arriba_izq, arriba_der, abajo_der, abajo_izq) = bbox
        arriba_izq = tuple([int(val) for val in arriba_izq])
        abajo_der = tuple([int(val) for val in abajo_der])
        
        # Delimitar solo las regiones con alta probabilidad y longitud de texto esperada
        if len(texto) >= 6 and len(texto) <= 10 :
            
            # Recortar la región detectada para que Tesseract la procese
            imagen_resultado = imagen_procesada[arriba_izq[1]:abajo_der[1], arriba_izq[0]+2:abajo_der[0]]

            return imagen_resultado, texto, probabilidad

    return imagen_procesada, '', 0.01

def procesar_deteccion(imagen_procesada):

    probabilidad_anterior = 0
    imagen_actual, texto_actual, probabilidad_actual = detectar_texto(imagen_procesada)
    

    while probabilidad_actual > probabilidad_anterior:
        probabilidad_anterior = probabilidad_actual
        imagen_anterior = imagen_actual
        texto_anterior = texto_actual
        print(f"Matrícula (EasyOCR): {texto_anterior}, Probabilidad: {probabilidad_anterior}")
        imagen_actual, texto_actual, probabilidad_actual = detectar_texto(imagen_anterior)


    return imagen_anterior, texto_anterior, probabilidad_anterior

    

def post_procesar_imagen(imagen_procesada):
    imagen_actual, texto_actual, probabilidad_actual = procesar_deteccion(imagen_procesada)
    probabilidad_anterior = 0
    imagen_anterior = ''
    texto_anterior = ''
    while probabilidad_actual > probabilidad_anterior:
        probabilidad_anterior = probabilidad_actual
        imagen_anterior = imagen_actual
        texto_anterior = texto_actual
        print("Procesando imagen...")
        post_procesado = procesar_imagen(imagen_actual)
        imagen_actual, texto_actual, probabilidad_actual = procesar_deteccion(post_procesado)
        mostrar_imagen(imagen_actual)
    
    return imagen_anterior, texto_anterior, probabilidad_anterior



if __name__ == "__main__":
    lector = easyocr.Reader(['en'], gpu=False)
    image_path = 'matriculas-europeas/val/UOU695.png'
    imagen = cv2.imread(image_path)
    mostrar_imagen(imagen)
    imagen_detectada, texto, probabilidad = post_procesar_imagen(imagen)
    
    if (probabilidad < 0.1):
        print("No se detectó ninguna texto")
    else:
        print(f"Resultado: {texto}, Probabilidad: {probabilidad}")





NameError: name 'easyocr' is not defined